# Multiforest optimization notebook

Above the code cells, there will be instructions how the users should modify the codes in the cells. If there are no instructions, then by default no changes should be needed for the cell.

## Specify input output

Define CC scenario data

In [ ]:
RCP = "RCP0"
filename = "rslt_inoutmetsaan_"+RCP+"_V13.zip"
sample = 0.1

Name definition for saved output, rule: _scenario_RCP_extension

In [ ]:
scenario = "MF"
extension = "test" # some additional info to the saved output

## Read .py class

In [ ]:
import multiFunctionalOptimization as MFO

In [ ]:
from importlib import reload
reload(MFO)

In [ ]:
mfo = MFO.MultiFunctionalOptimization(solver='CPLEX')

In [ ]:
import wget
import os
import pandas as pd
import numpy as np

## Read data

In [ ]:
if not filename in os.listdir("."):
    wget.download("https://a3s.fi/swift/v1/AUTH_9d5edfac7197434ab0e9b60b9f62c600/MF_opt_data/"+filename)

In [ ]:
%%time
mfo.readData(filename,
             # If no sample ratio given, the ratio is assumed to be 1
             sampleRatio= sample
             # Sample equally in all regions. 
             # Give the name of the column along the sampling should be equal (here region).             
             ,samplingSubsets = "region"
            )                 

* <b>NEW</b>: taking same sample among notebooks, see .py line 84 and 92
* Save the id of sampled stands to check if they are identical among notebooks

In [ ]:
id = mfo.data.id

# https://www.freecodecamp.org/news/python-unique-list-how-to-get-all-the-unique-values-in-a-list-or-array/
def get_unique_numbers(numbers):

    list_of_unique_numbers = []

    unique_numbers = set(numbers)

    for number in unique_numbers:
        list_of_unique_numbers.append(number)

    return list_of_unique_numbers

id = get_unique_numbers(id)
id = pd.DataFrame(id)
id.to_csv("./id_"+scenario+"_"+RCP+"_"+extension+"_18052021.csv")

## Create some new variables in the data

Calculate total (per stand) values from relative values

"Relative to Area" = indicator value relates 1 hectar -> scaled to represented area of NFI plot <br>
"Relative to volume" = indicator relates to standing V (e.g. %-share of deciduous trees) -> scaled to the represented volume of the plot <br>
"Absolute Value" = takes the inticator value as it is <br>

New columns are: <br>
* Total_... hectare value multiplied by represented area (or volume)<br>


In [ ]:
columnTypes = {
    'i_Vm3':(float,"Relative to Area"),
    'Harvested_V':(float,"Relative to Area"),
    'Harvested_V_log_under_bark':(float,"Relative to Area"), 
    'Harvested_V_pulp_under_bark':(float,"Relative to Area"),
    'Harvested_V_under_bark':(float,"Relative to Area"), 
    'Biomass':(float,"Relative to Area"),
    'ALL_MARKETED_MUSHROOMS':(float,"Relative to Area"), 
    'BILBERRY':(float,"Relative to Area"), 
    'COWBERRY':(float,"Relative to Area"),
    'HSI_MOOSE':(float,"Relative to Area"),
    'CAPERCAILLIE':(float,"Relative to Area"), 
    'HAZEL_GROUSE':(float,"Relative to Area"), 
    'V_total_deadwood':(float,"Relative to Area"), 
    'N_where_D_gt_40':(float,"Relative to Area"),
    'prc_V_deciduous':(float,"Relative to Area"),
    'CARBON_SINK':(float,"Relative to Area"), 
    'Recreation':(float,"Relative to Area"),
    'Scenic':(float,"Relative to Area")
}

In [ ]:
mfo.calculateTotalValuesFromRelativeValues(columnTypes=columnTypes)

List the new variables created:

In [ ]:
[name for name in mfo.data.columns if "Total_" in name and "Relative" not in name]

## Create new column:
1) Column indicating if regime is "CCF_3, CCF_4, BAUwGTR" (TRUE/FLASE) <br>
Important for ES Biodiversity, allowed regimes for conservation sites.

2) Column indicating if regime is "SA" (TRUE/FALSE)<br>
Important for ES Biodiversity, allowed regimes for statutory protection sites.

3) Column indicating if regime is "BAUwT_B, BAUwT_5_B, BAUwT_15_B, BAUwT_30_B, BAUwT_GTR_B" <br>
Important for ES Resillience, allowed regimes for climate change adaption.

4) Column indicating if regime is within all four CCF<br>
Important for ES Water under GLOBIOM V2 (enabled constraint -> soft target).

In [ ]:
regimeClassNames = {"regimeClass0name":"CCF",
                    "regimeClass1name":"SA",
                    "regimeClass2name":"Broadleave",
                    "regimeClass3name":"AllCCF"}
regimeClassregimes = {"regimeClass0regimes":["CCF_3","CCF_4","BAUwGTR"],
                      "regimeClass1regimes":["SA"],
                      "regimeClass2regimes":["BAUwT_B", "BAUwT_5_B", "BAUwT_15_B", "BAUwT_30_B", "BAUwT_GTR_B"],
                      "regimeClass3regimes":["CCF_1","CCF_2","CCF_3","CCF_4"]}

In [ ]:
mfo.addRegimeClassifications(regimeClassNames = regimeClassNames,regimeClassregimes=regimeClassregimes)

## New column for "soft target" of only CCF on peat (ES Water)

In [ ]:
# Column indicating if CCF and SA are on peat land - allowed regimes for water protection
mfo.data['CCFonPeat'] = np.where( 
    ( (mfo.data['AllCCF_forests'] == True ) & (mfo.data['PEAT'] == 1 ) ) | 
    ( (mfo.data['SA_forests'] == True) & (mfo.data['PEAT'] == 1 ) )
    , 1, 0
)

In [ ]:
# Create subsample to get the total peat area
peat = mfo.data[["id","represented_area_by_NFIplot","PEAT"]]
peat = peat[(peat["PEAT"] == 1)]
peat = peat.drop_duplicates(['id'])
totalPeat = peat["represented_area_by_NFIplot"].sum() / sample # divide by the sample ratio !!
totalPeat

In [ ]:
len(peat)

In [ ]:
# Column defining a peat stand´s area in relation to total peat area (USED for OPTIMIZATION!)
mfo.data['peatCCFArea'] = np.where(
    (mfo.data['CCFonPeat'] == 1 ), mfo.data['represented_area_by_NFIplot'] / totalPeat, 0
)

## Define initial value:
1) Define initial values; initial state is recognized by the regime "initial_state"

2) Create new variables that describe the <b>relative change to initial situation (start year) "Relative_"</b>:

In [ ]:
mfo.finalizeData(initialRegime="initial_state")

New variables created:

In [ ]:
[name for name in mfo.data.columns if "Relative_" in name]

In [ ]:
mfo.data.head()

In [ ]:
mfo.initialData.head()

## Start defining the optimization problem

<b>Objective format:</b>

Unique_key :[Long human readable name, column name in data, max/min objective, year wise aggregation, stand wise aggregation (, target year OR periodic targets)]

<b>Options for "objective":</b> "max"imise or "min"imise it <br>
<b>year wise aggregation:</b> "min" (minimum value), "average", "firstYear", "sum", "targetYearWithSlope","targetYear","lastYear", "periodicTargets" <br>
<b>stand wise aggregation:</b> "sum", "areaWeightedAverage", "areaWeightedSum" <br>
<b>targe year:</b> any year except the first one

## Group objective by ecosystem services

In [ ]:
wood_production = { 
    ## Previous policy objectives
    # Scenario BAU
    # max_TargetYearWithSlope_Sum_Objectives
    #"Total_i_Vm3_2025": ["Total annual timber volume increment by 2025 (m3)",
    #                     "Total_i_Vm3",
    #                     "max","targetYearWithSlope","sum",2025], 
    # Scenario BAU
    # max_TargetYearWithSlope_Sum_Objectives
    #"Total_i_Vm3_2050": ["Total annual timber volume increment by 2050 (m3)",
    #                     "Total_i_Vm3",
    #                     "max","targetYearWithSlope","sum",2050], 
    # Scenario BAU
    # max_TargetYearWithSlope_Sum_Objectives
    #"Total_Harvested_V_2025" :["Total annual harvested timber volume by 2025 (log & pulp) (m3)",
    #                           "Total_Harvested_V",
    #                           "max","targetYearWithSlope","sum",2025], 
    # Scenario Biodiv & Intens
    # max_Min_AreaWeightedAverage_Objectives
    #"Average_Harvested_V" : ["Average harvested timber volume (log & pulp) (m3/ha, evenflow)",
    #                         "Harvested_V",
    #                         "max","min","areaWeightedAverage"],
    
    ## Modified objective function(s) for multifunctionality assessment
    # Increment
    # used "areaWeightedSum" here for harmonizing among objectives
    # is however the same as using "Total_..." of an indicatore and "sum"
    # Both ways are leading to the same end     
    "Sum_i_Vm3": ["Annual timber volume increment(m3)",
                  "i_Vm3","max","min","areaWeightedSum"], 
    # Harvested volume
    "Sum_Harvested_V" :["Annual harvested timber volume (m3)",
                        "Harvested_V","max","min","areaWeightedSum"] 
}

In [ ]:
bioenergy = { 
    ## Previous policy objectives
    # Scenario BAU
    # max_TargetYearWithSlope_Sum_Objectives
    #"Total_Biomass_2025": ["Total annual harvested biomass volume by 2025 (m3)",
    #                       "Total_Biomass",
    #                       "max","targetYearWithSlope","sum",2025]

    ## Modified objective function(s) for multifunctionality assessment
    # Biomass
    "Sum_Biomass": ["Annual harvested biomass volume (m3)",
                    "Biomass","max","min","areaWeightedSum"]
}

In [ ]:
nonwood = { 
    ## Previous policy objectives
    # Scenario BAU
    # max_Min_Sum_Objectives
    #"Relative_BILBERRY": ["Bilberry yield (relative to 2016, max minimum over yrs)",
    #                      "Relative_Total_BILBERRY",
    #                      "max","min","sum"],
    #"Relative_COWBERRY": ["Cowberry yield (relative to 2016, max minimum over yrs)",
    #                      "Relative_Total_COWBERRY",
    #                      "max","min","sum"],
    #"Relative_ALL_MARKETED_MUSHROOMS": ["All marketed mushroom yield (relative to 2016, max minimum over yrs)",
    #                     "Relative_Total_ALL_MARKETED_MUSHROOMS",
    #                     "max","min","sum"]  
    
    ## Modified objective function(s) for multifunctionality assessment
    # Bilberries 
    "Sum_BILBERRY": ["Bilberry yield (kg)",
                     "BILBERRY","max","min","areaWeightedSum"],
    # Cowberries
    "Sum_COWBERRY": ["Cowberry yield (kg)",
                     "COWBERRY","max","min","areaWeightedSum"],
    # Mushrooms
    "Sum_ALL_MARKETED_MUSHROOMS": ["Marketed mushroom yield (kg)",
                                   "ALL_MARKETED_MUSHROOMS","max","min","areaWeightedSum"]    
}

In [ ]:
game = {
    ## Previous policy objectives
    # Scenario BAU & Biodiv
    # max_Average_sum_Objectives
    #"Sum_Total_HSI_MOOSE": ["Total habitat index for MOOSE (max average over all years)",
    #                       "Total_HSI_MOOSE",
    #                       "max","average","sum"],
    #"Sum_Total_HAZEL_GROUSE": ["Total habitat index for HAZEL_GROUSE (max average over yrs)",
    #                       "Total_HAZEL_GROUSE",
    #                       "max","average","sum"],
    #"Sum_Total_CAPERCAILLIE": ["Total habitat index for CAPERCAILLIE (max average over yrs)",
    #                       "Total_CAPERCAILLIE",
    #                       "max","average","sum"]    
    
    ## Modified objective function(s) for multifunctionality assessment
    # Moose
    # here used "average" instead "min" -> due to how the indices were designed
    "Sum_HSI_MOOSE": ["Habitat index for MOOSE (index)",
                      "HSI_MOOSE","max","average","areaWeightedSum"], 
    # Hazel Grouse
    "Sum_HAZEL_GROUSE": ["Habitat index for HAZEL_GROUSE (index)",
                         "HAZEL_GROUSE","max","average","areaWeightedSum"],
    # Capercaillie
    "Sum_CAPERCAILLIE": ["Habitat index for CAPERCAILLIE (index)",
                         "CAPERCAILLIE","max","average","areaWeightedSum"]    
}

In [ ]:
biodiversity = {
    ## Previous policy objectives
    # Scenario BAU
    # max_targetYear_areaWeightedAverage  (target values defined in (m3/ha)
    # ! "targetYear" instead "targetYearWithSlope", the latter doesn´t provide reasonable results
    # ! Reason: linear increase to target year causes troubles if zero/negative values exist
    #"Average_Deadwood_V_2025": ["Average Deadwood volume by 2025 (m3/ha)", 
    #                            "V_total_deadwood",
    #                            "max", "targetYear", "areaWeightedAverage", 2025], 
    # Scenario BAU
    # maximise the end value independent of initial situation
    # max_lastYear_sum_Objectives
    #"Total_N_where_D_gt_40": ["Total No. of trees diameter >= 40 cm  (max end value)",
    #                          "Total_N_where_D_gt_40",
    #                          "max","lastYear","sum"],    
    # Scenario BAU 
    # maximise the end value independent of initial situation
    # max_lastYear_sum_Objectives
    #"Total_prc_V_deciduous":  ["Total %-share of deciduous trees (related to V) (max end value)", 
    #                           "Total_prc_V_deciduous",
    #                           "max", "lastYear","sum"],
    
    ## Share of set aside - same objective fct. as in policy scenario 
    #"Ratio_SA_forests": ["Ratio of protected areas (%, SA forests)",
    #                     "SA_forests","max","firstYear","areaWeightedAverage"],   
    
    # Share of conservation management - same objective fct. as in policy scenario
    "Ratio_CCF_forests": ["Ratio of BC sites in commercial forests (%, CCF_3, CCF_4 and BAUwGTR)",
                          "CCF_forests","max","firstYear","areaWeightedAverage"],
    
    
    ## Modified objective function(s) for multifunctionality assessment
    # Deadwood
    "Sum_Deadwood_V": ["Deadwood volume (m3)", 
                       "V_total_deadwood","max","min","areaWeightedSum"], 
    # Deciduous trees (% of volume)
    "Sum_prc_V_deciduous":  ["Share of deciduous trees (% of standing volume)", 
                             "prc_V_deciduous","max", "min","areaWeightedSum"],
    # Large trees
    "Sum_N_where_D_gt_40": ["No. of large trees DBH >= 40 cm  (per ha)",
                            "N_where_D_gt_40","max","min","areaWeightedSum"]    
    
}

In [ ]:
climate_regulation = {
    ## Previous policy objectives
    # Scenario BAU
    # max_TargetYearWithSlope_Sum_Objectives
    #"Total_CARBON_SINK_2025": ["Total sequestration in carbon dioxide by 2025 (t CO2)",
    #                           "Total_CARBON_SINK",
    #                           "max","targetYearWithSlope","sum",2025] ,
    
    ## Modified objective function(s) for multifunctionality assessment
    # Carbon sink
    "Sum_CARBON_SINK": ["Sequestration of carbon dioxide (t CO2)",
                        "CARBON_SINK", "max","min","areaWeightedSum"] 
}

In [ ]:
recreation = {
    ## Previous policy objectives
    # Scenario BAU, Biodiv & Intens
    # max_min_sum_Objectives
    #"Sum_Total_Recreation" : ["Total Recreation index (max minimum over yrs)",
    #                          "Total_Recreation",
    #                          "max","min","sum"],
    #"Sum_Total_Scenic" : ["Total Scenic index (max minimum over yrs)",
    #                      "Total_Scenic",
    #                      "max","min","sum"]
    
    ## Modified objective function(s) for multifunctionality assessment
    # Recreation index
    "Sum_Recreation": ["Recreation index (-)",
                       "Recreation","max","min","areaWeightedSum"],
    # Scenic beauty index
    "Sum_Scenic": ["Scenic index (-)",
                   "Scenic","max","min","areaWeightedSum"]
}

In [ ]:
resilience = {
    # Share of Adaption to Climate Change (ACC) management - same objective fct. as in policy scenario
    "Ratio_ACC_forests": ["Ratio of adaptive management regimes (%, ..._B regimes)",
                          "Broadleave_forests","max","firstYear","areaWeightedAverage"]
}

In [ ]:
water = {
    # Share of CCF management on peat - same objective fct. as in policy scenario
    "Ratio_CCF_onPeat": ["Ratio of CCF on Peatland (%, all four CCF and SA)",
                         "peatCCFArea","max", "firstYear", "sum"]         
}

In [ ]:
objectives = {
              **wood_production,
              **bioenergy,
              **nonwood,
              **game,
              **biodiversity,
              **climate_regulation,
              **recreation,
              **resilience,
              **water
             }

In [ ]:
len(objectives)

In [ ]:
objectives.keys()

In [ ]:
mfo.data.columns

In [ ]:
[(col,mfo.data.dtypes[col]) for col in mfo.data.columns if "prc" in col]

## Define initial values if not available in data (initial_state)

Not NEEDED anymore?

Examples are increment, harvests, biomass and carbon sink. They are required for the "targetYearWithSlope" objective.

In [ ]:
#initialValues = {"Total_i_Vm3":107*10**6,               # value from National Forest Policy
#                 "Total_Harvested_V": 72.3*10**6,       # value from National Forest Policy
#                 "Total_Biomass": 2.9*10**6,            # value from National Forest Policy
#                 "Total_CARBON_SINK" : 34.1*10**6,      # value from National Forest Policy
#                                 
#                 "SA_forests" : 0.106,     # from ForestStatistics 2018
#                 "CCF_forests" : 0.015,    # from ForestStatistics 2018
#                 "BAUwGTR_forests":0.015}  # from ForestStatistics 2018

In [ ]:
#mfo.defineObjectives(objectives, initialValues = initialValues)
mfo.defineObjectives(objectives) 

## Define the constraints

Not NEEDED anymore?

In [ ]:
#CCFregimes = [regime for regime in mfo.regimes if "CCF" in regime] + ["SA"]

In [ ]:
#CCFregimes

Constraint format:
- Shortname: "constraint type","allowed regimes","human readable name",(regimes),"column in data")

In [ ]:
#constraintTypes = {"CCFonPeat":["Allowed regimes","Only CCF on peat lands",CCFregimes,"PEAT"]}

In [ ]:
#mfo.defineConstraints(constraintTypes)

## Calculate objective ranges

In [ ]:
mfo.data

In [ ]:
%%time
mfo.calculateObjectiveRanges(debug=True)

In [ ]:
mfo.objectiveRanges

# NEW - optimize for maximum Multifunctionality

Following Eyvindson et al. (2021). Now without a GUI, not needed.

## Define the ecosytem service categories

In [ ]:
ESS = {'wood_production':wood_production,
       'bioenergy':bioenergy,
       'nonwood':nonwood,
       'game':game,
       'biodiversity':biodiversity,
       'climate_regulation':climate_regulation,
       'recreation':recreation,
       'resilience':resilience,
       'water':water}

## Define how solutions by ESS are going to be aggregated

Based on the minimum value (MIN) or average value (AVG) for each ESS group. 
    
SOME LOGIC IS NEEDED FOR THIS. If there is only one objective in an ESS group, it doesn't matter. Has an ESS group multiple objectives, it does matter.

<b>Justification for using the MIN/AVG:</b> MIN is aiming for equity between the objectives, while using AVG is aiming for efficiency. E.g. Biodiversity, equity is likely more important than efficiency -- as who is to determine which species is more important than another ?

In [2]:
AGG = {'wood_production':"AVG",
       'bioenergy':"AVG",
       'nonwood':"AVG",
       'game':"MIN",
       'biodiversity':"MIN",
       'climate_regulation':"AVG",
       'recreation':"MIN",
       'resilience':"AVG",
       'water':"AVG"}
AGG

{'wood_production': 'AVG',
 'bioenergy': 'AVG',
 'nonwood': 'AVG',
 'game': 'MIN',
 'biodiversity': 'MIN',
 'climate_regulation': 'AVG',
 'recreation': 'MIN',
 'resilience': 'AVG',
 'water': 'AVG'}

## Ideal and anti-ideal value obtainable for each ESS category

In [3]:
mfo.addEyvindsonMultifunctionality(ESS,AGG)

NameError: name 'mfo' is not defined

## Maximum multifunctionality
    
The maximum that is achievable in each ESS category.

In [ ]:
mfo.solveMultifunctionality()

## Calculate some additional metrics of the latest calculated solution

#### Modifications in this section

You may add here visualizations and further calculations based on optimization if needed. These can be included in the GUI to be developed for the stakeholder meetings.

In [ ]:
regimeAmounts = {regime:0 for regime in mfo.regimes}
for key in mfo.regimesDecision.keys():
    regimeAmounts[key[1]] +=mfo.regimesDecision[key].solution_value()*mfo.standAreas.loc[key[0],"represented_area_by_NFIplot"]/mfo.standAreas["represented_area_by_NFIplot"].sum()

In [ ]:
%pylab notebook

In [ ]:
[val for val in regimeAmounts.values()]

In [ ]:
plt.plot([key for key in regimeAmounts.keys()],[val for val in regimeAmounts.values()])

In [ ]:
plt.bar(range(len(regimeAmounts)), list(regimeAmounts.values()), align='center')
plt.xticks(range(len(regimeAmounts)), list(regimeAmounts.keys()),rotation="vertical")

## Export data as csv

- <b>Solution_alldata</b> contains the optimal regime per stand AND the timely development of indicator values plus all other input columns (represented_are_by_NFIplot, region, NUTS2)
- <b>Solution</b> contains only the selected optimal regime and its share (if multiple regimes per stand are selected)

In [ ]:
try:
    os.mkdir("results")
except FileExistsError:
    pass
b = []
c = []
for key in mfo.regimesDecision.keys():
    if mfo.regimesDecision[key].solution_value() > 0:
        b = b+ [(key[0],x*5+2016, key[1]) for x in range(0,21)]
        c = c+ [(key[0],key[1],mfo.regimesDecision[key].solution_value())]
data2b = mfo.data.iloc[mfo.data.index.isin(b)]
data2b.to_csv("./results/solution_alldata_"+scenario+"_"+RCP+"_"+extension+".csv")
c1 = pd.DataFrame(c)
c1.to_csv("./results/solution_"+scenario+"_"+RCP+"_"+extension+".csv")

## Export objective ranges 

Can save re-calculation times if big data sets are optimisded <br>
<font color='red'>But, how to use it later for this purpose?</font>

In [ ]:
import json
mfo.objectiveRanges

with open('./results/objectiveRanges_'+scenario+'_'+RCP+'_'+extension+'.json', 'w') as json_file:
  json.dump(mfo.objectiveRanges, json_file)

Save the objective ranges also as CSV.

In [ ]:
df = pd.read_json('./results/objectiveRanges_'+scenario+'_'+RCP+'_'+extension+'.json')

df.to_csv('./results/objectiveRanges_'+scenario+'_'+RCP+'_'+extension+'.csv')

## Export objective values
The optimal solution for each objective.

In [ ]:
with open("./results/objectiveValues_"+scenario+'_'+RCP+'_'+extension+".csv","w") as file: 
    delim = "" 
    for objName in mfo.objectiveTypes.keys(): 
        file.write(delim+objName) 
        delim = "," 
    file.write("\n") 
    delim = "" 
    for objName in mfo.objectiveTypes.keys(): 
        file.write(delim+str(mfo.objective[objName].solution_value())) 
        delim = "," 
    file.write("\n")